In [31]:
import numpy as np
import pandas as pd
import ast
# Import linear_kernel
import nltk
import string
import re
import unidecode
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from collections import Counter
import config 
from sklearn.metrics.pairwise import linear_kernel
#Import TfIdfVectorizer (scikit-learn)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [89]:
df_82k = pd.read_csv ("C:/Users/Atif-Foxfire/Downloads/RAW_recipes.csv/recipes_82k.csv", sep="," )


In [90]:
df_82k.head()

,category,cooking_method,cuisine,image,ingredients,prep_time,recipe_name,serves,tags
0,NaN,['Set the racks in the middle and upper thirds...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['1 tablespoons extra virgin olive oil', '1 cu...",20 minutes,Mediterranean Sea Bass,4 servings,"Dairy Free, Gluten Free, Keto Recipes, Kid Fri..."
1,NaN,['Place the eggs in the air fryer basket and c...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['4 large eggs', 'Salt (black pepper, everythi...",15 minutes,Air Fryer Hard Boiled Eggs,4 eggs,"Air Fryer Recipes, Dairy Free, Gluten Free, Ke..."
2,NaN,"['Air Fryer directions:', 'Preheat air fryer t...",['American'],https://www.skinnytaste.com/wp-content/uploads...,"['olive oil spray', '4 about 5 ounce each salm...",5 minutes,Air Fryer Basil-Parmesan Salmon,4 servings,"Air Fryer Recipes, Gluten Free, Keto Recipes, ..."
3,NaN,"['Preheat the oven to 400F.', 'Pour 2 tablespo...",['American'],https://www.skinnytaste.com/wp-content/uploads...,['1/2 cup freshly grated Parmesan (not pre-gra...,15 minutes,Everything Parmesan Crisps,4 servings,"Gluten Free, Keto Recipes, Kid Friendly, Low C..."
4,NaN,['Cook potatoes in a large pot of salted water...,['American'],https://www.skinnytaste.com/wp-content/uploads...,['3 1/2 pounds new potatoes (about 10 peeled a...,10 minutes,Potato and Green Bean Salad,12 servings,"Dairy Free, Gluten Free, Kid Friendly, Vegetar..."


In [91]:
df_82k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82245 entries, 0 to 82244
Data columns (total 9 columns):
category          18107 non-null object
cooking_method    82245 non-null object
cuisine           82230 non-null object
image             82245 non-null object
ingredients       82245 non-null object
prep_time         58630 non-null object
recipe_name       82245 non-null object
serves            82244 non-null object
tags              82011 non-null object
dtypes: object(9)
memory usage: 5.6+ MB


In [92]:
print('No of Unique recipes: ',len(df_82k['recipe_name'].unique().tolist()))
print('No of Unique Ingredients: ', len(df_82k['ingredients'].unique().tolist()))

No of Unique recipes:  55838
No of Unique Ingredients:  61198


In [93]:
df_82k = df_82k.drop_duplicates(subset=['ingredients'], keep='first')
df_82k.reset_index(drop=True, inplace=True)

In [94]:
df_82k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61198 entries, 0 to 61197
Data columns (total 9 columns):
category          8442 non-null object
cooking_method    61198 non-null object
cuisine           61189 non-null object
image             61198 non-null object
ingredients       61198 non-null object
prep_time         43163 non-null object
recipe_name       61198 non-null object
serves            61197 non-null object
tags              60992 non-null object
dtypes: object(9)
memory usage: 4.2+ MB


In [95]:
type(df_82k['ingredients'])
total = df_82k.isnull().sum().sort_values(ascending=False)
percent = (df_82k.isnull().sum()/df_82k.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
category,52756,0.862054
prep_time,18035,0.294699
tags,206,0.003366
cuisine,9,0.000147
serves,1,0.000016


In [96]:
 df_82k.drop('category',1, inplace =True)

In [97]:
vocabulary = nltk.FreqDist()
# This was done once I had already preprocessed the ingredients
for ingredients in df_82k['ingredients']:
    ingredients = ingredients.split()
    vocabulary.update(ingredients)
for word, frequency in vocabulary.most_common(200):
    print(f'{word};{frequency}')

'1;190164
cup;130832
'2;106743
tablespoons;69816
'1/2;66981
teaspoon;65919
and;65375
cups;47154
'1/4;44691
ground;40677
tablespoon;39803
fresh;37142
'3;33130
for;32251
pepper',;31901
or;30791
1/2;30220
'4;28977
oil',;28168
chopped;27993
black;27700
salt',;26669
freshly;25274
chopped',;24064
ounces;23226
sugar',;22342
olive;21910
large;21862
teaspoons;21339
to;20700
into;19545
red;18893
cut;18382
['1;17793
finely;17344
plus;16472
pound;15245
'Kosher;14053
white;13523
of;13293
unsalted;13067
1;12740
butter',;12346
salt;11877
flour',;11750
['2;11613
small;11574
'Salt;11569
garlic,;11504
sliced;11420
powder',;11405
'6;11344
'3/4;11015
grated;10989
sliced',;10709
chicken;10642
onion,;10543
butter,;10315
cloves;10246
'8;10063
all-purpose;9879
pounds;9848
juice',;9693
(about;9547
2;9459
vinegar',;9231
cream',;9073
medium;8989
thinly;8975
leaves',;8918
vanilla;8911
peeled;8900
recipe;8649
sauce',;8621
dried;8127
green;8009
'1/3;7953
water',;7946
extra-virgin;7940
kosher;7932
whole;7842
lemon;7

In [98]:
def ingredient_parser(ingreds):
    '''
    
    '''
    measure_units = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', 'tbl.', 'tb', 'tbsp.', 'fluid ounce', 'fl oz', 'gill', 'cup', 'c', 'pint', 'p', 'pt', 'fl pt', 'quart', 'q', 'qt', 'fl qt', 'gallon', 'g', 'gal', 'ml', 'milliliter', 'millilitre', 'cc', 'mL', 'l', 'liter', 'litre', 'L', 'dl', 'deciliter', 'decilitre', 'dL', 'bulb', 'level', 'heaped', 'rounded', 'whole', 'pinch', 'medium', 'slice', 'pound', 'lb', '#', 'ounce', 'oz', 'mg', 'milligram', 'milligramme', 'g', 'gram', 'gramme', 'kg', 'kilogram', 'kilogramme', 'x', 'of', 'mm', 'millimetre', 'millimeter', 'cm', 'centimeter', 'centimetre', 'm', 'meter', 'metre', 'inch', 'in', 'milli', 'centi', 'deci', 'hecto', 'kilo']
    words_to_remove = ['everything','about','each','chopped','diced','together','fresh', 'oil', 'a', 'red', 'bunch', 'and', 'clove', 'or', 'leaf', 'chilli', 'large', 'extra', 'sprig',
    'ground', 'handful', 'free', 'small', 'pepper', 'virgin', 'range', 'from', 'dried', 'sustainable', 'black', 'peeled', 'higher',
                       'welfare', 'seed', 'for', 'finely', 'freshly', 'sea', 'quality', 'white', 'ripe', 'few', 'piece', 'source', 'to',
                       'organic', 'flat', 'smoked', 'ginger', 'sliced', 'green', 'picked', 'the', 'stick', 'plain', 'plus', 'mixed', 'mint',
                       'bay', 'basil', 'your', 'cumin', 'optional', 'fennel', 'serve', 'mustard', 'unsalted', 'baby', 'paprika', 'fat', 'ask',
                       'natural', 'skin', 'roughly', 'into', 'such', 'cut', 'good', 'brown', 'grated', 'trimmed', 'oregano', 'powder', 'yellow',
                       'dusting', 'knob', 'frozen', 'on', 'deseeded', 'low', 'runny', 'balsamic', 'cooked', 'streaky', 'nutmeg', 'sage', 'rasher',
                       'zest', 'pin', 'groundnut', 'breadcrumb', 'turmeric', 'halved', 'grating', 'stalk', 'light', 'tinned', 'dry', 'soft', 'rocket',
                       'bone', 'colour', 'washed', 'skinless', 'leftover', 'splash', 'removed', 'dijon', 'thick', 'big', 'hot', 'drained', 'sized',
                       'chestnut', 'watercress', 'fishmonger', 'english', 'dill', 'caper', 'raw', 'worcestershire', 'flake', 'cider', 'cayenne',
                       'tbsp', 'leg', 'pine', 'wild', 'if', 'fine', 'herb', 'almond', 'shoulder', 'cube', 'dressing', 'with', 'chunk', 'spice', 'thumb',
                       'garam', 'new', 'little', 'punnet', 'peppercorn', 'shelled', 'saffron', 'other','chopped', 'salt', 'olive', 'taste', 'can', 'sauce',
                       'water', 'diced', 'package', 'italian', 'shredded', 'divided', 'parsley', 'vinegar', 'all', 'purpose', 'crushed', 'juice', 'more',
                       'coriander', 'bell', 'needed', 'thinly', 'boneless', 'half', 'thyme', 'cubed', 'cinnamon', 'cilantro', 'jar', 'seasoning', 'rosemary',
                       'extract', 'sweet', 'baking', 'beaten', 'heavy', 'seeded', 'tin', 'vanilla', 'uncooked', 'crumb', 'style', 'thin', 'nut', 'coarsely',
                       'spring', 'chili', 'cornstarch', 'strip', 'cardamom', 'rinsed', 'honey', 'cherry', 'root', 'quartered', 'head', 'softened', 'container',
                       'crumbled', 'frying', 'lean', 'cooking', 'roasted', 'warm', 'whipping', 'thawed', 'corn', 'pitted', 'sun', 'kosher', 'bite', 'toasted', 'lasagna',
                       'split', 'melted', 'degree', 'lengthwise', 'romano', 'packed', 'pod', 'anchovy', 'rom', 'prepared', 'juiced', 'fluid', 'floret', 'room', 'active',
                       'seasoned', 'mix', 'deveined', 'lightly', 'anise', 'thai', 'size', 'unsweetened', 'torn', 'wedge', 'sour', 'basmati', 'marinara', 'dark',
                       'temperature', 'garnish', 'bouillon', 'loaf', 'shell', 'reggiano', 'canola', 'parmigiano', 'round', 'canned', 'ghee', 'crust', 'long',
                       'broken', 'ketchup', 'bulk', 'cleaned', 'condensed', 'sherry', 'provolone', 'cold', 'soda', 'cottage', 'spray', 'tamarind', 'pecorino',
                       'shortening', 'part', 'bottle', 'sodium', 'cocoa', 'grain', 'french', 'roast', 'stem', 'link', 'firm', 'asafoetida', 'mild', 'dash', 'boiling','one','two','three','four']
    # The ingredient list is now a string so we need to turn it back into a list. We use ast.literal_eval
    if isinstance(ingreds, list):
        ingredients = ingreds
    else:
        ingredients = ast.literal_eval(ingreds)
    # We first get rid of all the punctuation. We make use of str.maketrans. It takes three input 
    # arguments 'x', 'y', 'z'. 'x' and 'y' must be equal-length strings and characters in 'x'
    # are replaced by characters in 'y'. 'z' is a string (string.punctuation here) where each character
    #  in the string is mapped to None. 
    translator = str.maketrans('', '', string.punctuation)
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for i in ingredients:
        i.translate(translator)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', i)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        items = [unidecode.unidecode(word) for word in items] #''.join((c for c in unicodedata.normalize('NFD', items) if unicodedata.category(c) != 'Mn'))
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        items = [word for word in items if word not in measure_units]
        # Get rid of common easy words
        items = [word for word in items if word not in words_to_remove]
        if items:
            ingred_list.append(' '.join(items)) 
    ingred_list = " ".join(ingred_list)
    return ingred_list

In [99]:
df_82k['parsed_ingredients'] = df_82k['ingredients'].apply(lambda x: ingredient_parser(x))



In [100]:
df_82k.head()

,cooking_method,cuisine,image,ingredients,prep_time,recipe_name,serves,tags,parsed_ingredients
0,['Set the racks in the middle and upper thirds...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['1 tablespoons extra virgin olive oil', '1 cu...",20 minutes,Mediterranean Sea Bass,4 servings,"Dairy Free, Gluten Free, Keto Recipes, Kid Fri...",onion garlic tomato their wine kalamata chilea...
1,['Place the eggs in the air fryer basket and c...,['American'],https://www.skinnytaste.com/wp-content/uploads...,"['4 large eggs', 'Salt (black pepper, everythi...",15 minutes,Air Fryer Hard Boiled Eggs,4 eggs,"Air Fryer Recipes, Dairy Free, Gluten Free, Ke...",egg bagel
2,"['Air Fryer directions:', 'Preheat air fryer t...",['American'],https://www.skinnytaste.com/wp-content/uploads...,"['olive oil spray', '4 about 5 ounce each salm...",5 minutes,Air Fryer Basil-Parmesan Salmon,4 servings,"Air Fryer Recipes, Gluten Free, Keto Recipes, ...",salmon lemon mayonnaise like sir parmesan cheese
3,"['Preheat the oven to 400F.', 'Pour 2 tablespo...",['American'],https://www.skinnytaste.com/wp-content/uploads...,['1/2 cup freshly grated Parmesan (not pre-gra...,15 minutes,Everything Parmesan Crisps,4 servings,"Gluten Free, Keto Recipes, Kid Friendly, Low C...",parmesan pre using hole box sesame minced onio...
4,['Cook potatoes in a large pot of salted water...,['American'],https://www.skinnytaste.com/wp-content/uploads...,['3 1/2 pounds new potatoes (about 10 peeled a...,10 minutes,Potato and Green Bean Salad,12 servings,"Dairy Free, Gluten Free, Kid Friendly, Vegetar...",potato bean bean scallion


In [107]:
#df_82k['parsed_ingredients'] =       df_recipes.ingredients_parsed.values.astype('U')
# TF-IDF feature extractor
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(df_82k['parsed_ingredients'])
tfidf_recipe = tfidf.transform(df_82k['parsed_ingredients'])

In [114]:
tfidf_recipe.shape

(61198, 8824)

In [108]:
input_ingredients = ['Tomato', 'onion', 'potato', 'beef', "rice"]

In [109]:
# parse the ingredients using my ingredient_parser
try:
    ingredients_parsed = ingredient_parser(input_ingredients)
except:
    ingredients_parsed = ingredient_parser([input_ingredients])
# use our pretrained tfidf model to encode our input ingredients
input_ingredients_tfidf = tfidf.transform([ingredients_parsed])

# calculate cosine similarity between actual recipe ingreds and input ingreds
cos_sim = map(lambda x: cosine_similarity(input_ingredients_tfidf, x), tfidf_recipe)
scores = list(cos_sim)

In [110]:
# getting top 5 recomendations
top_score = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]
recommendation_df = pd.DataFrame(columns = ['recipe_name', 'ingredients','cooking_method','cuisine', 'score' ])

count = 0
for i in top_score:
    recommendation_df.at[count, 'recipe_name'] = (df_82k['recipe_name'][i])
    recommendation_df.at[count, 'ingredients'] =0(df_82k['ingredients'][i])
    recommendation_df.at[count, 'cooking_method'] = df_82k['cooking_method'][i]
    recommendation_df.at[count, 'cuisine'] = df_82k['cuisine'][i]
    recommendation_df.at[count, 'score'] = "{:.3f}".format(float(scores[i]))
    count += 1


In [111]:
recommendation_df.head()

,recipe_name,ingredients,cooking_method,cuisine,score
0,Stuffed Peppers,"['1/2 cup rice', '1 1/2 cups water', '1/4 cup ...",['In a 2 cup measuring cup combine rice and wa...,[],0.772
1,Porcupine Balls,"['1 pound lean ground beef', '1 1/2 cups uncoo...","['Preheat the oven to 350 degrees F.', 'Put th...",[],0.637
2,Southside Chili,"['2 to 3 tablespoons oil', '5 pounds lean grou...",['For Southside Chili: In a large pot over med...,[],0.623
3,Moroccan Meatballs,"['For the sauce', '1 onion (chopped)', '2 14-o...","['Slow Cooker Directions:', 'Place the onions ...",['American'],0.602
4,Portuguese Rice,"['1/4 cup olive oil', '1 1/2 cups finely chopp...",['In a 4-quart medium saucepan heat the oil ov...,[],0.599


In [118]:
recommendation_df['ingredients'][1]

"['1 pound lean ground beef', '1 1/2 cups uncooked white rice, divided', '1 teaspoon dried onion', '1 teaspoon seasoned salt', '1 teaspoon Italian seasoning', '1 (14.5-ounce) can diced tomatoes with juice', '1 (14-ounce) can beef broth']"

In [119]:
recommendation_df['cooking_method'][1]

'[\'Preheat the oven to 350 degrees F.\', \'Put the ground beef into a mixing bowl. Measure 1/2 cup of the rice and put it in with the meat. Mix it all together with your clean hands. Form the mixture into 8 or 9 golf ball-size balls.\', \'In a large casserole dish, add the remaining rice, dried onion, salt, Italian seasoning, diced tomatoes and beef broth. Stir together with the spoon and top with the meatballs.\', \'Cover the dish tightly with foil and bake for about 1 hour. The rice in the meatballs will have puffed up all around the meat, and your meatballs will look like little porcupines!\', "Cook\'s Note: Let an adult help you remove the dish from the oven with oven mitts or hot pads. When you remove the foil, steam will come out, so be careful!"]'